In [1]:
import numpy as np
import torch
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import unicodedata
import re
import os
import io
import time

In [2]:
import csv

with open("rus.txt", "r", encoding="utf-8") as f:
    reader = csv.reader(f, delimiter="\t")
    df = pd.DataFrame([row for row in reader], columns=[n for n in range(3)])

df.head()

,0,1,2
0,Go.,Марш!,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Иди.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,Идите.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Hi.,Здравствуйте.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
4,Hi.,Привет!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


In [3]:
df = df.drop(2, axis=1)

df.head()


,0,1
0,Go.,Марш!
1,Go.,Иди.
2,Go.,Идите.
3,Hi.,Здравствуйте.
4,Hi.,Привет!


In [4]:
df.shape

(392305, 2)

In [5]:
df = df.rename(columns={0: "output", 1: "input"})
df.tail()

,output,input
392300,"In today's world, we have to equip all our kid...",В современном мире перед нами стоит задача дат...
392301,Death is something that we're often discourage...,"Смерть - это зачастую то, разговоры или даже м..."
392302,"At a moment when our economy is growing, our b...","В тот момент, когда наша экономика растёт, наш..."
392303,Since there are usually multiple websites on a...,"Поскольку сайтов, посвящённых какой-либо теме,..."
392304,Doubtless there exists in this world precisely...,"Несомненно, для каждого мужчины в этом мире гд..."


In [6]:
data = df.loc[:50000, :]
data.head()

,output,input
0,Go.,Марш!
1,Go.,Иди.
2,Go.,Идите.
3,Hi.,Здравствуйте.
4,Hi.,Привет!


In [7]:
# Remove the row with the value 33851 from the train DataFrame
# Remove the row with the value 33851 from the train DataFrame
data.isnull().any()

output    False
input     False
dtype: bool

In [8]:
data.shape

(50001, 2)

In [9]:
# Remove the row with the value 33851 from the DataFrame
data.loc[50000, ['input', 'output']]

input          Как твой отец?
output    How is your father?
Name: 50000, dtype: object

In [10]:
print(data.index)

RangeIndex(start=0, stop=50001, step=1)


In [11]:
# Split the data into train and test sets
train = data.sample(frac=0.8, random_state=42)
test = data.drop(train.index)

In [12]:
print(train.index)

Index([33553,  9427,   199, 12447, 39489, 42725, 10822, 49499,  4144, 36958,
       ...
        7523, 38598, 48799, 42041,  1559, 13313, 13528, 25017, 19317,  7928],
      dtype='int64', length=40001)


In [13]:
# Reset the indexes of the train and test DataFrames
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [14]:
print(train.index, '\n', test.index)

RangeIndex(start=0, stop=40001, step=1) 
 RangeIndex(start=0, stop=10000, step=1)


In [15]:
train.head()

,output,input
0,Tom is improving.,Том исправляется.
1,He's not home.,Он не дома.
2,Come on!,Пошли!
3,Tom was irked.,Том был раздражён.
4,I need to eat now.,Сейчас мне нужно поесть.


In [34]:
import torch
import transformers
import pandas as pd
from transformers import AutoTokenizer, Trainer, TrainingArguments
from torch.utils.data import Dataset

class CustomDataset(Dataset):

    def __init__(self, df, input_column, output_column):
        """
        Args:
            df: A Pandas DataFrame containing the training or test data.
            input_column: The name of the column in the DataFrame that contains the input text.
            output_column: The name of the column in the DataFrame that contains the output text.
        """
        self.df = df
        self.input_column = input_column
        self.output_column = output_column
        self.tokenizer = AutoTokenizer.from_pretrained('facebook/bart-base')

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_text = self.df[self.input_column][idx]
        output_text = self.df[self.output_column][idx]
        input_ids = self.tokenizer(input_text, max_length=256, padding='max_length', truncation=True)['input_ids']
        output_ids = self.tokenizer(output_text, max_length=256, padding='max_length', truncation=True)['input_ids']
        return {'input_ids': input_ids, 'output_ids': output_ids}

In [22]:
#tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


In [23]:
# Tokenize the data
#train['input_ids'] = train['input'].apply(tokenizer.encode_plus)
#train['output_ids'] = train['output'].apply(tokenizer.encode_plus)
#test['input_ids'] = test['input'].apply(tokenizer.encode_plus)
#test['output_ids'] = test['output'].apply(tokenizer.encode_plus)

In [24]:
print(transformers.__version__)

4.30.1


In [38]:
# Create the model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-base')

In [39]:
train_dataset = CustomDataset(train, 'input', 'output')


test_dataset = CustomDataset(test, 'input', 'output') 

In [42]:
# Create the Trainer object
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    logging_steps=100,
    evaluation_strategy="epoch",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
    #eval_dataset=test_dataset
    
)

# Train the model
trainer.train()

RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 411770880 bytes.